## Item 1: Know which Version of Python You're Using

우선 아주 간단한 팁으로 시작합니다. 사용하고 있는 Python 버전을 확인하라는 것인데요. 


In [ ]:
import sys
print(sys.version_info)

In [ ]:
print(sys.version)

위와 같은 명령으로 사용중인 Python 버전을 확인할 수 있습니다.   
또한 가능한 Python3를 사용하라는 충고가 있습니다.

## Item 2: Follow the PEP 8 Style Guide

PEP(Python Enhancement Proposal)에는 파이썬에 대한 개선 제안, 구조에 대한 디자인 등 다양한 문서가 포함되어 있는데요. 그중에서도 8번 문서(<a href="https://www.python.org/dev/peps/pep-0008/">PEP8</a>)에는 코딩 스타일 가이드를 담고 있습니다. 


.

## Item 14 : Prefer Exceptions to Returning None

함수 구현할 때 어떤 특이상태를 표현하기 위해 None을 return하도록 만들곤 한다.      
그런데 이런 방식은 return 값에 대한 상태체크시 (잘 인지하지 못하는) 에러를 유발하곤 한다.     
예를들어 아래 코드는 0으로 나누는 경우를 체크하려 하지만 분자가 0인 경우에도 'Invalid inputs'를 출력한다.      
(if 문과 관련하여 아래 ※ if if if... (Item 14)를 참고)

In [57]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

In [58]:
x, y = 0, 5
result = divide(x, y)
if not result:
    print('Invalid inputs')
else:
    print(result)

Invalid inputs


In [59]:
x, y = 1, 0
result = divide(x, y)
if result is None:
    print('Invalid inputs')
else:
    print(result)

Invalid inputs


이에 대한 첫번째 대안으로 이상 상태가 발생했는지 아닌지를 표현하는 flag를 원래 반환하려는 값과 함께 tuple로 반환하는 것이다.

In [60]:
def divide(a, b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None

In [61]:
x, y = 1, 0
success, result = divide(x, y)
if not success:
    print('Invalid inputs')
else:
    print(result)

Invalid inputs


In [62]:
x, y = 0, 5
success, result = divide(x, y)
if not success:
    print('Invalid inputs')
else:
    print(result)

0.0


(보다 나은) 두번째 대안으로 함수에서 에러가 발생하면 그냥 에러를 raise해서 호출한쪽에서 에러를 처리하게 하는 것이다. 

In [63]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs') from e

In [65]:
x, y = 1, 0
try:
    result = divide(x, y)
except ValueError:
    print('Invalid inputs')
else:
    print('Result is %.1f' %result)

Invalid inputs


In [66]:
x, y = 0, 5
try:
    result = divide(x, y)
except ValueError:
    print('Invalid inputs')
else:
    print('Result is %.1f' %result)

Result is 0.0


## Item 15 : Know How Closures Interact with Variable Scope

주어진 숫자들을 정렬하되, 특별한 그룹에 속하는 숫자들은 그렇지 않은 숫자들보다 우선하도록 정렬하는 함수를 만들고 싶다.
예를들어 정렬할 숫자로 8, 3, 1, 2, 5, 4, 7, 6가 주어지고, 특별한 그룹으로 2, 3, 5, 7가 주어지는 경우 아래와 같은 결과를 얻고 싶다.    
2, 3, 5, 7, 1, 4, 6, 8   
이런 함수를 아래와 같이 작성해볼 수 있다.

In [78]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)

In [79]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


위 함수 정의에서 살펴볼 것은 아래 3가지이다.
- 위 helper 함수는 closure이다. (helper 함수는 local scope 밖의 변수인 group에 접근)
- Python에서 함수는 first class object이다. (함수를 인자로 전달하거나, 함수의 결과로 반환하거나, 변수에 할당할 수 있다.)
- tuple간의 대소 비교는 0번째 원소부터 순차적으로 진행된다. (아래 코드 참고)

In [80]:
# tuple의 비교 연산은 0번째 원소부터 진행됨
print((1, 2, 3) > (1, 2, 4))
print((1, 2, 3) > (2, 2, 3))

False
False


숫자들을 전달한 후 group에 속하는 숫자들이 등장 했는지 확인하기 위해 아래와 같이 found 함수를 추가해볼 수 있다.

In [81]:
def sort_priority2(values, group):
    found = False
    def helper(x):
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found

In [82]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
is_found = sort_priority2(numbers, group)
print(numbers, ',', is_found)

[2, 3, 5, 7, 1, 4, 6, 8] , False


그런데 분명 numbers의 숫자들 중 group에 속하는 값이 있음에도 False가 출력되고 있다. 이는 변수를 읽을 때와 쓸 때 scope을 판단하는 기준이 달라서 종종 범하게 되는 문제이다.

변수를 읽을 때에는 1. 현재 함수의 scope, 2. 함수 밖의 scope, 3. 해당 코드가 포함된 모듈, 4. built-in scope 을 차례로 찾는다.    
반면 변수를 쓸 때에는 현재 scope에 선언된 변수가 있다면 그것에 기록할 것이고, 만약 없다면 현재 scope에 새로운 변수를 선언고 기록한다. 즉 변수를 쓸 때에는 현재 scope밖을 살피지 않는다. 또한 주의할 점은 변수를 읽는 것과 쓰는 것이 한 scope안에 함께 있다면 그 변수는 local 변수로 간주되며, 이때 만약 변수를 읽는 것이 먼저 등장한다면 (변수 할당 전에 변수 값에 접근하므로) 에러가 발생한다.

위 문제를 해결하려면 어떻게 해야할까?

해결책은 두가지인데 우선 첫번째는 Python3에서 추가된 nonlocal 키워드를 사용하는 것이다.      
즉 local scope에서 해당 변수는 local 변수가 아니다라는 것을 명시하는 것으로서,       
enclosing scope(local scope 바로 밖)에 선언된 해당 변수를 찾아 사용하게 된다.

In [83]:
def sort_priority3(values, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found

In [84]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
is_found = sort_priority3(numbers, group)
print(numbers, ',', is_found)

[2, 3, 5, 7, 1, 4, 6, 8] , True


두번째 해결책은 scope 밖에 있는 변수를 list, dictionary, set혹은 직접 정의한 class와 같은 mutable type을 사용하고, 이 변수의 멤버 함수를 이용하여 값을 읽거나 쓰는 것이다. (found[0] = True는 사실 found.`__setitem__`(True) 로 동작한다.) 

In [85]:
def sort_priority4(values, group):
    found = [None]
    def helper(x):
        if x in group:
            found[0] = True
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found[0]

In [86]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
is_found = sort_priority4(numbers, group)
print(numbers, ',', is_found)

[2, 3, 5, 7, 1, 4, 6, 8] , True


그런데 그런데 그런데...

아주 단순한 경우가 아니면 가능한한 nonlocal를 사용하지 않는 것을 추천한다. 구조가 복잡해지면 어떤 변수를 참고하고 있는 것인지 찾기가 어려워지기 때문이다. 부득이 복잡한 구조에서 nonlocal 과 같은 변수 사용이 필요하다면 아래와 같이 inner function을 class로, nonlocal 변수를 클래스의 멤버 변수로 바꾸면 가독성을 높일 수 있다.

In [87]:
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False
    
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)

In [88]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sorter = Sorter(group)
numbers.sort(key=sorter)
print(numbers, ',', sorter.found)

[2, 3, 5, 7, 1, 4, 6, 8] , True


---

## if if if ...

아래와 같이 x값을 테스트 하는 함수가 있고

In [ ]:
def test_if(x):
    if x:
        print('O', end='')
    else:
        print('X', end='')
    
    if x is True:
        print('O', end='')
    else:
        print('X', end='')
        
    if x == True:
        print('O', end='')
    else:
        print('X', end='')

x가 ﻿2, 1, 0, -1, -2, None, '', True 인 각각의 경우 출력값은?

In [22]:
test_if(2)

OXX

In [23]:
test_if(1)

OXO

In [24]:
test_if(0)

XXX

In [25]:
test_if(-1)

OXX

In [26]:
test_if(-2)

OXX

In [27]:
test_if(None)

XXX

In [28]:
test_if('')

XXX

In [29]:
test_if(True)

OOO

- `if x:`는 x가 0, None, ''이 아니면 True.    
- `if x is True`는 x가 True일때만 True.    
- `if x == True`는 x가 1이나 True이면 True.

## Metaclass

.

## Decorator

Decorator는 어떤 함수를 입력으로 받아 이 함수에 전/후 처리를 더하거나 전혀 새로운 함수를 반환하는 '호출 가능한 어떤 것'(callable)을 말한다. 이 callable은 함수(decorator 함수) 형태로 정의되고, 대상 함수 정의앞에 @{decorator 함수}를 명시하는 형태로 사용된다.

아래 코드에서 decorator 함수의 이름은 deco이고, 대상함수명은 'target'이며 이 함수 정의 앞에 '@deco'를 명시하고 있다. 즉 target 함수는 deco 함수의 인자로 입력되고 inner라는 새로운 함수로 바뀌고 있다.

In [35]:
def deco(func):
    print('Decorating func:', func)
    
    def inner():
        print('Inner function')
        
    return inner

@deco
def target():
    print('Running target()')

decorated_target = target

Decorating func: <function target at 0x000001C90812A840>


In [38]:
decorated_target()

Inner function


사실 위 코드는 아래 코드와 같으며, 함수를 decorator 함수로 양념/포장/데코하는 것이다.

In [39]:
def deco(func):
    print('Decorating func:', func)
    
    def inner():
        print('Inner function')
        
    return inner

def target():
    print('Running target()')

decorated_target = deco(target)

Decorating func: <function target at 0x000001C9080FABF8>


In [40]:
decorated_target()

Inner function


위와 같이 명시적으로 함수를 인자로 하는 함수를 호출하는 것과 Decorator의 다른점은 decorator를 사용할 경우 decorator 함수 내부에서의 처리는 타겟 하수가 정의된 시점(import 시점, loading 시점)이라는 것이다. 물론 대상 함수(위 예에서는 target함수)가 실행되는 시점은 그 함수가 호출되는 시점(runtime)이다.

실제 구현에서는 decorator 함수와 대상 함수가 다른 모듈에서 정의되는 경우가 많으며, 입력받은 함수와 전혀 다른 함수를 반환하는 경우가 대부분이다. 물론 입력 받은 함수와 동일한 함수를 반환하는 경우도 있으나 이런 것은 보통 웹 프레임웍에서 URL 패턴을 등록하는 것과 같이 특정 목적의 함수들을 관리하려 할 때 사용된다.

## Closures

Closure는 자신이 정의하지 않은 nonglobal 변수를 포함하는 함수를 말한다. (a closure is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available... / Fluent Python, p201)

예를들어 아래 'printer'는 inner_printer에서 정의하지 않은 (outer_printer의 local)변수 x를 포함하고 있으며, 호출시에는 y값만 인자로 전달한다. 

변수 x는 outer_printer의 scope에 존재하므로 'return inner_printer' 이후 이 scope은 존재하지 않고 x도 사라지는 것처럼 보이지만, 이 변수는 inner_printer에서 **사용**(읽기)되면서 이 함수 정의에 포함되어 전달된다.

주의할점은 **사용**이 읽기가 아니라 쓰기일 경우 그 의미가 크게 달라지는데.. 이 부분은 나중에 설명하기로 한다.

In [114]:
def outer_printer(x):
    def inner_printer(y):
        print('closure variable:', x)
        print('local variable  :', y)
        
    return inner_printer

In [115]:
printer = outer_printer(x=100)

In [116]:
printer(y=200)

closure variable: 100
local variable  : 200


closure는 함수의 `__closure__`변수를 통해 접근할 수 있다.

In [117]:
print(printer.__closure__)
print(printer.__closure__[0].cell_contents)

(<cell at 0x000001C90810BB28: int object at 0x0000000056B9F630>,)
100


또한 `__code__.co_freevars`를 통해 closure 변수의 이름을 확인할 수 있다.

In [118]:
printer.__code__.co_freevars

('x',)

참고로 함수의 local 변수는 아래와 같이 `__code__.co_varnames`로 확인할 수 있다.

In [121]:
printer.__code__.co_varnames

('y',)

참고로2 앞서 예제 코드는 아래와 같이 구현할수도 있다.

In [125]:
class OuterPrinter:
    def __init__(self, x):
        self.x = x
        
    def __call__(self, y):
        print('closure variable:', self.x)
        print('local variable  :', y)

In [126]:
printer = OuterPrinter(x=100)

In [127]:
printer(200)

closure variable: 100
local variable  : 200


참고 페이지 : https://www.programiz.com/python-programming/closure

## Variable Scope rule

아래 함수 f1에서 선언된 지역변수 a는 성공적으로 그 값이 출력된다. 그리고 변수 b는 선언되어 있지 않으므로 에러가 발생된다.

In [1]:
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

아래와 같이 함수 f1 밖에서 (전역변수) b를 선언하고, 다시 실행하면 b가 정상적으로 출력된다.

즉 변수를 읽을 때에는 우선 현재 **함수의 범위(scope)**를 찾고 없으면 차례로 **함수 밖의 범위**, **해당 코드가 포함된 모듈(=global scope)**, **built-in 범위**에서 해당 이름의 변수를 찾는다.

In [2]:
b = 6
f1(3)

3
6


---

그런데 아래 코드를 보면    
함수 f1에서는 변수 b를 읽으려 하고    
함수 f2에서는 변수 b에 값을 할당하려 하고 있다.

3이 출력된 것으로 보아 (지역)변수 a를 읽는데에는 문제가 없었으나, 변수 b를 print하려 할 때 문제가 발생한다.     
왜 에러가 발생한 걸까?

그 이유는 함수 f2가 컴파일될 때 함수내 변수들의 성격을 미리 정의하는데, 변수 b는 'b = 9'와 같이 할당이 일어나기 때문에 **지역 변수**로 결정된다. 이후 실행과정에서 'print(b)'에서 변수 b값을 읽어와야 하는데, 지역변수 b는 아직 값이 할당되지 않았기 때문에 에러가 발생한다.

물론 할당하는 것 조차 함수 밖의 범위들을 찾아보고 같은 이름의 변수가 있다면 그 변수값을 수정하도록 할 수 있으나 알수 없는 범위의 변수가 수정되는 것을 막기 위한 Python의 설계적 선택이라 할 수 있다.(javascript에서는 지역변수가 없을 경우 외부 범위의 변수를 수정한다.)

즉 변수 할당에서는 함수 내 범위의 변수에만 적용된다.

In [21]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

In [22]:
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

물론 종종 변수 밖의 변수값을 수정할 필요가 있을 수 있다. 이때 사용하는 것이 global 키워드이다.   
(혹은 global 키워드 없이 list같은 mutable 변수를 이용하는 방법도 있다.)

In [18]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

In [19]:
f3(3)

3
6


In [20]:
b

9

## Item 16: Consider Generators Instead of Returning Lists

어떤 목록을 반환하는 함수를 만들 때 list를 반환하기보다는 generator를 사용하는 방법이 더 좋을 수 있다.     
예를들어 어떤 문장에서 단어가 시작되는 index들 알고자 한다고 해보자.    
우선 간단하게는 아래와 같이 index들을 list 형태로 반환하는 형식으로 구현할 수 있다.   

In [28]:
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        if letter == ' ':
            result.append(index + 1)
    return result

In [31]:
result = index_words('Four score and seven years ago')
print(result)

[0, 5, 11, 15, 21, 27]


그런데 위와 같은 방식의 함수 구현은 몇가지 문제점을 갖고 있다.

첫번째 문제는 코드 가독성이 떨어지고 지저분하다는 것이다. 빈칸이 보이면 index+1을 전달하는 것이 핵심인데, 위 코드에서는 result list에 append하는 등의 작업이 더 부각되어 보인다.     
이런 방식의 구현보다는 아래와 같이 generator를 사용하는 권장한다. (generator는 yield를 이용해 결과를 반환하는 함수를 의미함)

In [32]:
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

In [33]:
result = index_words('Four score and seven years ago')
print(result)

[0, 5, 11, 15, 21, 27]


두번재 문제는 list 반환으로 구현할 경우 반환할 list의 크기에 따라 out of memory가 발생할 수 있다는 점이다.    
반면 generator를 사용할 경우 한번 반환(yield)하는 값의 크기만큼의 메모리로 충분하다.    
예를들어 파일을 읽어 단어가 시작되는 index들을 알고자 한다면 아래와 같이 generator를 이용해 구현할 수 있다.

In [1]:
import os

In [2]:
os.getcwd()

'D:\\_PlayGround\\Github\\pg\\dev_python_data_eng'

In [36]:
def index_file(handle):
    offset = 0
    for line in handle:
        if line:
            yield offset  # Starting index of line
        for letter in line:
            offset += 1
            if letter == ' ':
                yield offset

In [37]:
from itertools import islice

file_path = './data/data.txt'
with open(file_path, 'r') as f:
    it = index_file(f)
    results = islice(it, 0, 10)
    print(list(results))

[0, 4, 5, 15, 27, 30, 34, 43, 44, 45]
